# Урок 5

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from utils import prefilter_items

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import FeatureUnion

import seaborn as sns

from sklearn.preprocessing import QuantileTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [6]:
data = pd.read_csv('retail.csv', sep=';')
data.columns = [col.lower() for col in data.columns]
data['customerid'] = data['customerid'].fillna(-1)
data['customerid'] = data['customerid'].astype('int32')
data.rename(columns={'customerid': 'user_id'},
           inplace=True)
len(data['description'].unique())

4224

In [8]:
data = pd.read_csv('../data/retail_train.csv')

item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [9]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [10]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [11]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [12]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [13]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [14]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [15]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [16]:

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [17]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [18]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [19]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
user_feat.shape

(2499, 7)

In [21]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [22]:
item_feat.shape

(86865, 6)

## Encoding features

In [23]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [24]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Домашнее задание

1) Прочитать статьи про BPR, WARP loss

2) Сделать грид серч текущей модели

# Pipeline

In [44]:
#Гибридные рекоммендательные системы 
class HybridRecommendationSystem(BaseEstimator, TransformerMixin):
    def __init__(self, model, hyperParam):
        self.model = model.
        self.hyperParam = hyperParam

    def fit(self, X, y=None):
        
        self.model.fit((sparse_user_item > 0) * 1,
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15,
          num_threads=4,
          verbose=False)
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [ ]:
class HyperParam:
    def __init__(self, no_components,loss, learning_rate, item_alpha, user_alpha, random_state, epochs):
        self.no_components = no_components
        self.loss = loss
        self.learning_rate = learning_rate
        self.item_alpha = item_alpha
        self.user_alpha = user_alpha
        self.random_state = random_state
        self.epochs = epochs

In [ ]:
class Model:
    def __init__(self, sparse_user_item, sample_weight, user_features, item_features, num_threads, verbose):
        self.sparse_user_item = sparse_user_item
        self.sample_weight = sample_weight
        self.user_features = user_features
        self.item_features = item_features
        self.num_threads = num_threads
        self.verbose = verbose

In [58]:
%%time
dic = dict()

for no_comp in [60,40,20,5]:
    for ls in ['bpr']:
        for lr in [0.05, 0.01, 0.005]:
            for i_al in [0.01]:
                for u_al in [0.5]:
                    model = LightFM(no_components=no_comp,
                                             loss=ls,
                                             learning_rate=lr, 
                                             item_alpha=i_al,
                                             user_alpha=u_al, 
                                             random_state=42)
                        
                    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                                      sample_weight=coo_matrix(user_item_matrix),
                                      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                      epochs=15,
                                      num_threads=4,
                                      verbose=False)

                    pred = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
                    dic[f'{no_comp}_{ls}_{lr}_{i_al}_{u_al}'] = pred
                    print(f'{no_comp}_{ls}_{lr}_{i_al}_{u_al} = {pred}')

60_bpr_0.05_0.01_0.5 = 0.0002400960511295125
60_bpr_0.01_0.01_0.5 = 0.0018407364841550589
60_bpr_0.005_0.01_0.5 = 0.0008003201219253242
40_bpr_0.05_0.01_0.5 = 0.0013605442363768816
40_bpr_0.01_0.01_0.5 = 0.0013605442363768816
40_bpr_0.005_0.01_0.5 = 0.0006402560975402594
20_bpr_0.05_0.01_0.5 = 0.0
20_bpr_0.01_0.01_0.5 = 0.0008003201219253242
20_bpr_0.005_0.01_0.5 = 0.0006402560975402594
5_bpr_0.05_0.01_0.5 = 0.0
5_bpr_0.01_0.01_0.5 = 0.0
5_bpr_0.005_0.01_0.5 = 8.003201219253242e-05
Wall time: 3h 59min 12s


In [59]:
%%time
dic = dict()

for no_comp in [60,40,20,5]:
    for ls in ['warp']:
        for lr in [0.05, 0.01, 0.005]:
            for i_al in [0.1]:
                for u_al in [0.1]:
                    model = LightFM(no_components=no_comp,
                                             loss=ls,
                                             learning_rate=lr,
                                             item_alpha=i_al,
                                             user_alpha=u_al,
                                             random_state=42)
                        
                    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                                      sample_weight=coo_matrix(user_item_matrix),
                                      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                      epochs=15,
                                      num_threads=4,
                                      verbose=False)

                    pred = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
                    dic[f'{no_comp}_{ls}_{lr}_{i_al}_{u_al}'] = pred
                    print(f'{no_comp}_{ls}_{lr}_{i_al}_{u_al} = {pred}')

60_warp_0.05_0.1_0.1 = 0.05170067772269249
60_warp_0.01_0.1_0.1 = 0.05122048780322075
60_warp_0.005_0.1_0.1 = 0.05122048780322075
40_warp_0.05_0.1_0.1 = 0.05170067772269249
40_warp_0.01_0.1_0.1 = 0.05122048780322075
40_warp_0.005_0.1_0.1 = 0.05122048780322075
20_warp_0.05_0.1_0.1 = 0.05170067772269249
20_warp_0.01_0.1_0.1 = 0.05122048780322075
20_warp_0.005_0.1_0.1 = 0.05122048780322075
5_warp_0.05_0.1_0.1 = 0.05170067772269249
5_warp_0.01_0.1_0.1 = 0.05122048780322075
5_warp_0.005_0.1_0.1 = 0.05122048780322075
Wall time: 3h 23min 45s


In [ ]:
dic

Вывод. Для алгоритма BPG количество компонентов, скорость обучения и альфа значения имеют большое значение для точности значения.
Для WARP скорость обучения не влияет на точность модели. Необходимо проверить альфа значения.